In [1]:
import os
import pandas as pd
import torch

from src.datasets import ContentWise, RL4RS, OpenCDP
from src.utils import get_train_val_test_tmatrix_tnumitems

pkl_path = '../pkl/'
data_path = '../data/recsys/'

# ContentWise

In [10]:
c_path = os.path.join( data_path, 'ContentWiseImpressions/data/ContentWiseImpressions/CW10M-CSV/')

c = ContentWise(c_path, min_session_len=3, max_session_len=100)
c.dump(os.path.join(pkl_path, 'cw.pkl'))

reading files. . . 
preprocessing. . . 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 329870/329870 [00:11<00:00, 29745.69it/s]


biulding affinity matrix...


272907it [01:49, 2490.35it/s]


### Self-check

In [ ]:
BATCH_NO = 0

# loading pickled dataset, gettin data loaders
c = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
( 
    c_train_loader, 
    c_val_loader, 
    c_test_loader, 
    c_train_user_item_matrix, 
    train_num_items,
) = get_train_val_test_tmatrix_tnumitems(c, batch_size=2, seed=123)

# get batch from test loader
i = 0
for batch in c_test_loader:
    if i < BATCH_NO:
        i += 1
        continue
    break

for key, value in batch.items():
    print(f'\n == {key} ==')
    print(value)

# get users
user_id = batch['user_ids'][0][0].item(), batch['user_ids'][1][0].item()
user_idx = batch['user_indexes'][0][0].item(), batch['user_indexes'][1][0].item()



In [ ]:
# response counts user 1: carefully check manually
for i, user in enumerate(user_idx):
    print(f'\nuser {user}\n')
    print(c_train_user_item_matrix[user, :])
    items = batch['slates_item_indexes'][i][batch['responses'][i] > 0]
    counts = batch['responses'][i][batch['responses'][i] > 0]
    print(torch.stack([items, counts]).T)

# batches are collated OK

In [ ]:
# checking dataset: search random user
USER_ID = 1223
DATA_POINT = -1
for i, user in enumerate(c):
    if user['user_ids'][0]==USER_ID:
        DATA_POINT = i
c[DATA_POINT]['slates_item_ids'], c[DATA_POINT]['responses']

In [ ]:
# manually compare with tables
ratings = pd.read_csv(
    os.path.join(c_path, 'interactions.csv.gz'), 
    compression='gzip', 
    delimiter=','
)
ratings[(ratings.user_id == USER_ID) & (ratings.recommendation_id != -1)].sort_values('utc_ts_milliseconds')

In [ ]:
c.metadata[c.metadata['user_id'] == USER_ID]

In [ ]:
c.responses[c.metadata[c.metadata['user_id'] == USER_ID].index]

In [ ]:
slates_log = pd.read_csv(
    os.path.join(c_path, 'impressions-direct-link.csv.gz'),
    compression='gzip', 
    delimiter=','
)

In [ ]:
recommendations = ratings[(ratings.user_id == USER_ID) & (ratings.recommendation_id != -1)].recommendation_id.unique()
slates_log[slates_log.recommendation_id.isin(recommendations)]

# RL4RS

In [ ]:
r_path = os.path.join(data_path, 'rl4rs-dataset/')
r = RL4RS(
    r_path,
    which='rl4rs_dataset_b_sl.csv', 
    min_session_len=3
)
r.dump(os.path.join(pkl_path, 'rl4rs.pkl'))

In [ ]:
r = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
DATA_POINT = 1234
session = r[DATA_POINT]['session']
for key, value in r[DATA_POINT].items():    
    print(f'\n == {key} ==')
    print(value)


In [ ]:
df = pd.read_csv(
    os.path.join(r_path,'rl4rs_dataset_b_sl.csv'),
    delimiter='@'
)
df[df['session_id'] == session ]

 # OpenCDP

In [ ]:
cosmetics_path = os.path.join(data_path, 'OpenCDP/cosmetics')
for w_size in [5, 10, 20]:
    for a_hrs in [1, 2, 4, 8, 24]:
        o = OpenCDP(
            cosmetics_path, 
            min_session_len=3 * w_size, # early crop sessions with < 3 user-recsys iteractions
            window_size=w_size, 
            max_active_hours=a_hrs,
        )
        o.dump(os.path.join(pkl_path, f'cosmetics_{w_size}_{a_hrs}.pkl'))

### It is too large to handle it simultaneously -- using only first month

In [ ]:
multi_path = os.path.join(data_path, 'OpenCDP/multi') 
for w_size in [5, 10, 20]:
    for a_hrs in [1, 2, 4, 8, 24]:
        o = OpenCDP(
            multi_path,
            files = ['2019-Dec.csv'],
            min_session_len=3 * w_size, # early crop sessions with < 3 user-recsys iteractions
            window_size=w_size, 
            max_active_hours=a_hrs,
        )
        o.dump(os.path.join(pkl_path, f'multi_{w_size}_{a_hrs}.pkl'))

In [ ]:
data_dir = multi_path
files = None 

files = os.listdir(data_dir) if files is None else files
events = [pd.read_csv(os.path.join(data_dir, file)) for file in files]
raw_events = pd.concat(events)

# there are separate sessions for single user,
raw_events.user_id.count(), raw_events.user_session.count()

In [ ]:
raw_events.event_type.value_counts()

In [ ]:
# чтобы собрать хоть какие-то выдачи, нам надо хотя-бы несколько десятков событий для каждого юзера
c = raw_events[["user_session","event_time"]].groupby('user_session').count()
c[(c.event_time < 300) & (c.event_time > 30)].plot.hist()

In [ ]:
# практически все сессии из одного взаимодействия, придется их дропнуть. 
MIN_SESSION_LEN = 30
good_sessions = c[c.event_time > 30].index
print(len(good_sessions))
raw_events = raw_events[raw_events.user_session.isin(good_sessions)]
raw_events.head()

In [ ]:
# cart > view wtf?
raw_events.event_type.value_counts()

**Из чата:**
1. Берем в качестве "рекомендаций" просмотры, а в качестве "отклика"— добавления в корзину. Конверсия из корзины в покупки — тоже можно, но звучит гораздо менее естественно, основной вариант первый.

2. Объединяем наши "рекомендации" в выдачи двумя вариантами:
* либо по дням, типа **"выдача" —  это все просмотренные за день товары**
* либо окном фиксированно длины. **"выдача" — это последние M просмтренных айтемов**
* либо (подумал уже после обсуждения) объединяя оба варианта, то есть "выдачей" считаются те айтемы, которые просмотрены не слишком много времени назад, и при этом не слишком много айтемов назад

3. Пробуем все варианты, играем размером окна/периодом/etc, смотрим при каком получаются самые адекватные размеры "выдач" и "сессий"

Все это, конечно, в каком-то смысле манипуляция данными, но на безрыбье (при отсутсвии других датасетов с явно залогированными выдачами), выбирать не приходится. Заняться этим планируется уже после формального дедлайна, так как у нас есть два датасета с явными выдачами, и мы считаем метрики на них. Потом просто добавится чисел на новых данных.

In [ ]:

separator = 'fixed' # period

# фиксированное окно из k предметов
WINDOW_SIZE = 8
# число часов, в течение которых клики понимаются как отдельная "выдача"
ACTIVE_HOURS = 24

recommendations = []
responses = []
metadata = []
idx = 0   

for session, group in tqdm(raw_events.groupby('user_session')):
    rec = []
    clicks = []
    for timestamp, row in group.sort_values(by='event_time').iterrows():
        if row.event_type == 'remove_from_cart' or row.event_type == 'purchase':
            continue
        
        # сохраняем время первого взаимодействия в "выдаче"
        if len(rec) == 0:
            first_action = datetime.strptime(row.event_time, "%Y-%m-%d %H:%M:%S UTC")
        # добавляем событие просмотра /  корзины
        if row.event_type == 'cart':
            if row.product_id not in rec:
                rec.append(row.product_id)
                clicks.append(1)
            else:
                clicks[rec.index(row.product_id)] += 1
        elif row.event_type == 'view':
            rec.append(row.product_id)
            clicks.append(0)
        else:
            raise ValueError

        # ecли прошло слишком много времени внутри выдачи, закончим ее
        if (datetime.strptime(row.event_time, "%Y-%m-%d %H:%M:%S UTC") -
                first_action >
                timedelta(hours=ACTIVE_HOURS)):
            rec.extend([-1] * (WINDOW_SIZE - len(rec)))
            clicks.extend([0] * (WINDOW_SIZE - len(clicks)))
            
        if len(rec) == WINDOW_SIZE:
            metadata.append({
                'recommendation_idx': len(recommendations),
                'session_id': row.user_session,
                'user_id': row.user_id,
                'timestamp': row.event_time,    
            })
            recommendations.append(rec)
            responses.append(clicks)
            rec, clicks = [], []

In [ ]:
rr = np.array(responses)
rr.sum(-1).mean()

In [ ]:
from src.datasets import RecommendationData
r = RecommendationData(
            recommendations = np.array(recommendations).astype(int),
            responses = np.array(responses).astype(int),
            metadata = pd.DataFrame(metadata)
        )


In [ ]:
df = pd.DataFrame(metadata)
df

In [ ]:
for i in range(len(r)):
    if -1 in r[i]['slates_item_ids']:
        break
r[i]
    

In [ ]:
raw_events[(raw_events.user_session == '0002dda7-653c-44b3-be14-ff8857e90f1f')].sort_values('event_time').head(50)

# Generated Dataset statistics

In [ ]:
res = {}

for filename in os.listdir('../pkl'):
    if not (
        # filename.startswith('multi') or\
        filename.startswith('cosmetics')
    ):
        continue
    o = OpenCDP.load(
        os.path.join('../pkl/', filename),
    )
    o.metadata.set_index(['session_id', 'timestamp'], drop=False, inplace=True)
    o.metadata.sort_index(inplace=True)
    o.dump(os.path.join('../pkl/', filename))
    N = len(o)
    stats = {
        'length': 0,
        'real_slate_size': 0        
    }
    for datapoint in tqdm(o, desc=filename):
        stats['length'] += datapoint['length']
        stats['real_slate_size'] += datapoint['slates_mask'].sum()
    stats['real_slate_size'] /= stats['length']
    stats['length'] /= N
    res[filename] = stats
res